## Demonstration of Materials Project Energy Corrections

This notebook illustrates how to apply and obtain an explanation of energy corrections used in the Materials Project database.

Author: Ryan Kingsbury

Date: May 2021

### Overview

The Materials Project API (`MPRester`) returns `ComputedEntry` objects that contain information about DFT calculations. By default, these objects have adjustments applied to the energies of certain elements to reduce certain systematic errors. See our [documentation](https://docs.materialsproject.org/methodology/total-energies/#total-energy-adjustments) for complete details.

As of Spring 2021, `ComputedEntry` are processed using the `MaterialsProject2020Compatibility` class in pymatgen by default. The legacy correction scheme, used from 2010-2020, is still available in `MaterialsProjectCompatibility`.

In [ ]:
# Uncomment the subsequent lines in this cell to install dependencies for Google Colab.
# !pip install pymatgen==2022.7.19

In [ ]:
from pymatgen.entries.compatibility import (
    MaterialsProject2020Compatibility,
    MaterialsProjectCompatibility,
)
from pymatgen.ext.matproj import MPRester

### Default behavior - `MaterialsProject2020Compatibility`

Let's retrieve entries in the `Cl-Mo-O` system to demonstrate how this works.

In [ ]:
# retrieve
with MPRester() as m:
    entries = m.get_entries_in_chemsys("Cl-Mo-O")
entry = entries[0]

You can examine the energy corrections via the `energy_adjustments` attribute 
of the `ComputedEntry`. This attribute contains a list of each energy correction that has been applied.

In [ ]:
entries[25].energy_adjustments

[]

If you want even more detail, you can examine an indiviual `EnergyAdjustment` (one element of the list)

### Applying the legacy corrections with `MaterialsProjectCompatibility`

If you want to use the old corrections, or apply your own, you can re-process the `ComputedEntry` obtained from `MPRester` using a `Compatibility` class. The `.process_entries` method will remove any previously-applied energy corrections and re-process the entry in-place.

In [ ]:
compat = MaterialsProjectCompatibility()

entries = compat.process_entries(entries)

In [ ]:
entries[25].energy_adjustments

[ConstantEnergyAdjustment:
   Name: MP Anion Correction
   Value: -2.809 eV
   Uncertainty: nan eV
   Description: Constant energy adjustment (-2.809 eV)
   Generated by: MaterialsProjectCompatibility,
 ConstantEnergyAdjustment:
   Name: MP Advanced Correction
   Value: -28.248 eV
   Uncertainty: nan eV
   Description: Constant energy adjustment (-28.248 eV)
   Generated by: MaterialsProjectCompatibility]

Notice how the energy adjustments have changed. The class name, description and values are all different. You will also notice that the descriptions of the legacy corrections are less verbose than those of the modern `MaterialsProject2020Compatibility` corrections.

### Removing corrections altogther

If you want to remove all corrections from a `ComputedEntry`, simply set `energy_adjustments` to an empty list. You can verify that you have removed corrections by checking the `energy_per_atom` and the `correction_per_atom` of the `ComputedEntry` before and after.

In [ ]:
entries[25].energy_per_atom

-4.639152145227273

In [ ]:
entries[25].correction_per_atom

-0.7058445454545454

In [ ]:
entries[25].energy_adjustments = []
entries[25].energy_per_atom

-3.9333075997727276

In [ ]:
entries[25].correction_per_atom

0.0

Alternatively, you can simply pass `compatible_only=False` to the `MPRester` call when you download data.

In [ ]:
# retrieve
with MPRester() as m:
    entries = m.get_entries_in_chemsys("Cl-Mo-O", compatible_only=False)
entry = entries[0]

In [ ]:
entries[25].energy_adjustments

[]